In [1]:
import os
import cv2
import ipywidgets.widgets as widgets
import threading
import time
from IPython.display import display
import numpy as np
from Smart_Car import pid, servo

# 设置摄像头显示组件
image_widget = widgets.Image(format='jpeg', width=640, height=480)

# bgr8转jpeg格式
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# 打开摄像头
cap = cv2.VideoCapture(0)

# 检查摄像头是否成功打开
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# 设置图像宽度和高度
width = 640
height = 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# 设置工作目录为项目根目录
os.chdir(r'/home/asus/Project/')

# 加载预训练的SSD模型
modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt"

# 检查文件是否存在
if not os.path.exists(modelFile):
    print(f"Error: {modelFile} does not exist.")
else:
    print(f"{modelFile} exists.")

if not os.path.exists(configFile):
    print(f"Error: {configFile} does not exist.")
else:
    print(f"{configFile} exists.")

try:
    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
except Exception as e:
    print(f"Failed to load network: {e}")
    exit()

# 初始化舵机
pan_pid = pid.PID(kP=0.77, kI=0.45, kD=0.3)
tilt_pid = pid.PID(kP=0.77, kI=0.45, kD=0.3)
pan_pid.initialize()
tilt_pid.initialize()

# 初始角度
pan_angle = 90
tilt_angle = 90
servo.set_servo_angle(servo.SERVO_CHANNEL_PAN, pan_angle)
servo.set_servo_angle(servo.SERVO_CHANNEL_TILT, tilt_angle)

# 设置死区阈值（例如，图像宽度和高度的1%）
DEADZONE_WIDTH = 0.01
DEADZONE_HEIGHT = 0.01

# 设置最大角度增量
MAX_ANGLE_INCREMENT = 5

# 初始化当前舵机角度
current_pan_angle = 90
current_tilt_angle = 90

def camera():
    global current_pan_angle,current_tilt_angle
    try:
        display(image_widget)  # 显示摄像头组件
        while True:
            ret, frame = cap.read()
            if not ret:
                print("无法读取帧")
                break

            # 获取图像的高度和宽度
            h, w = frame.shape[:2]

            # 创建一个blob对象
            blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

            # 设置输入blob到网络
            net.setInput(blob)

            # 进行前向传播以获取预测结果
            detections = net.forward()

            face_center_x = None
            face_center_y = None

            # 遍历检测结果
            for i in range(detections.shape[2]):
                confidence = detections[0, 0, i, 2]

                # 过滤掉低置信度的结果
                if confidence > 0.5:
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")

                    # 绘制矩形框标记人脸
                    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)

                    # 计算人脸中心位置
                    face_center_x = (startX + endX) // 2
                    face_center_y = (startY + endY) // 2

                    # 只跟踪第一个识别出的人脸
                    break

            if face_center_x is not None and face_center_y is not None:
                # 计算误差
                pan_error = (face_center_x - w // 2) / (w // 2)
                tilt_error = (face_center_y - h // 2) / (h // 2)

                # 检查是否在死区内
                if abs(pan_error) < DEADZONE_WIDTH and abs(tilt_error) < DEADZONE_HEIGHT:
                    # 如果在死区内，不调整舵机
                    continue

                # 更新PID控制器
                pan_output = pan_pid.update(pan_error)
                tilt_output = tilt_pid.update(tilt_error)

                # 计算新的舵机角度增量
                pan_angle_increment = max(-MAX_ANGLE_INCREMENT, min(MAX_ANGLE_INCREMENT, int(pan_output * MAX_ANGLE_INCREMENT)))
                tilt_angle_increment = max(-MAX_ANGLE_INCREMENT, min(MAX_ANGLE_INCREMENT, int(tilt_output * MAX_ANGLE_INCREMENT)))

                # 更新当前舵机角度
                current_pan_angle = max(0, min(180, current_pan_angle - pan_angle_increment))
                current_tilt_angle = max(0, min(180, current_tilt_angle + tilt_angle_increment))

                # 设置舵机角度
                servo.set_servo_angle(servo.SERVO_CHANNEL_PAN, current_pan_angle)
                servo.set_servo_angle(servo.SERVO_CHANNEL_TILT, current_tilt_angle)

            # 更新图像组件
            image_widget.value = bgr8_to_jpeg(frame)
            time.sleep(0.02)
    except KeyboardInterrupt:
        print("程序关闭！")
        cap.release()

# 创建并启动线程
camera_thread = threading.Thread(target=camera)
camera_thread.start()

res10_300x300_ssd_iter_140000.caffemodel exists.
deploy.prototxt exists.


Image(value=b'', format='jpeg', height='480', width='640')